<hr style="border:2px solid gray"> </hr>

# MachineLearning
<hr style="border:2px solid gray"> </hr>

# Step 0. Import packages and Global Settings

In [5]:
import os, glob, numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
from keras import backend as K

import tensorflow.compat.v1 as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

data_name = 'image_data.npy'
category_file_name = 'Categories.txt'
model_dir = './model'

# Step 1. Data Loading

In [2]:
X_train, X_test, y_train, y_test = np.load(data_name, allow_pickle=True)

X_train = X_train.astype(float) / 255
X_test = X_test.astype(float) / 255

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)


(7947, 64, 64, 3) (7947, 100)
(2650, 64, 64, 3) (2650, 100)


In [3]:
import codecs
category_file = codecs.open(category_file_name, 'r', 'utf-8')
categories = list(map(str.strip, category_file.readlines()))
classes = len(categories)

# Step 2. Define Model

In [6]:
model = Sequential()
model.add(Conv2D(32, (3,3), padding="same", input_shape=X_train.shape[1:], activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
    
model.add(Conv2D(64, (3,3), padding="same", activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))
    
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
if not os.path.exists(model_dir):
    os.mkdir(model_dir)
    
model_path = model_dir + '/model.model'
checkpoint = ModelCheckpoint(filepath=model_path , monitor='val_loss', verbose=1, save_best_only=True)
early_stopping = EarlyStopping(monitor='val_loss', patience=6)

In [7]:
model.summary()

Model: &quot;sequential_1&quot;
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 16384)  

# Training

In [8]:
history = model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_test, y_test), callbacks=[checkpoint, early_stopping])
print("정확도 : %.4f" % (model.evaluate(X_test, y_test)[1]))

Epoch 1/50
248/249 [============================&gt;.] - ETA: 0s - loss: 4.2481 - accuracy: 0.0478
Epoch 00001: val_loss improved from inf to 3.98583, saving model to ./model\model.model
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./model\model.model\assets
249/249 [==============================] - 20s 79ms/step - loss: 4.2473 - accuracy: 0.0479 - val_loss: 3.9858 - val_accuracy: 0.1049
Epoch 2/50
248/249 [============================&gt;.] - ETA: 0s - loss: 3.8229 - accuracy: 0.1066
Epoch 00002: val_loss improved from 3.98583 to 3.69006, saving model to ./model\model.model
INFO:tensorflow:Assets written to: ./model\model.model\assets
249/249 [==============================] - 19s 78ms/step - loss: 3.8230 - accuracy: 0.1066 - val_loss: 3.6901 - val_accuracy: 0.164

# Test

In [9]:
K.backend()

&#39;tensorflow&#39;